In [16]:
import numpy as np
from data import get_fixed_data
from PriceProcess import price_model
from WindProcess import wind_model

def simulate_experiment(policy, E=10):
    """
    Simulates E independent experiments to evaluate a given decision-making policy.
    :param policy: A function defining the decision-making policy.
    :param E: Number of independent experiments (days).
    :return: Average cost over E experiments.
    """
    # Retrieve fixed parameters from the data file
    data = get_fixed_data()
    T = data['num_timeslots']
    D = data['demand_schedule']
    C = data['hydrogen_capacity']
    P2H = data['p2h_max_rate']
    H2P = data['h2p_max_rate']
    R_p2h = data['conversion_p2h']
    R_h2p = data['conversion_h2p']
    C_elzr = data['electrolyzer_cost']
    
    total_costs = []
    
    for _ in range(E):
        # Initialize wind power and electricity price
        wind_power = np.zeros(T)
        price = np.zeros(T)
        
        wind_power[0] = data['target_mean_wind']
        price[0] = data['mean_price']
        
        if T > 1:
            wind_power[1] = wind_power[0]
            price[1] = price[0]
        
        for t in range(2, T):
            wind_power[t] = wind_model(wind_power[t-1], wind_power[t-2], data)
            price[t] = price_model(price[t-1], price[t-2], wind_power[t], data)
        
        # Initialize system state
        hydrogen_storage = np.zeros(T+1)  # Hydrogen storage level
        grid_power = np.zeros(T)  # Power taken from the grid
        p2h = np.zeros(T)  # Power-to-Hydrogen
        h2p = np.zeros(T)  # Hydrogen-to-Power
        electrolyzer_status = np.zeros(T, dtype=int)  # Whether the electrolyzer is on
        
        total_cost = 0
        
        # Simulate the time steps
        for t in range(T):
            electrolyzer_status[t], p2h[t], h2p[t], grid_power[t] = policy(t, hydrogen_storage[t], wind_power[t], price[t], data)
            
            # Ensure hydrogen storage respects bounds
            hydrogen_storage[t+1] = min(C, max(0, hydrogen_storage[t] + R_p2h * p2h[t] - h2p[t] / R_h2p))
            
            # Compute the cost at this time step
            total_cost += price[t] * grid_power[t] + C_elzr * electrolyzer_status[t]
        
        total_costs.append(total_cost)
    
    return np.mean(total_costs)

def dummy_policy(t, h_level, wind, price, data):
    """
    Dummy policy: Never use the electrolyzer.
    :return: (E_ON, p2h, h2p, grid_power)
    """
    D = data['demand_schedule'][t]
    return 0, 0, 0, max(0, D - wind)  # No hydrogen conversion, only use grid if necessary

# Run evaluation on dummy policy
E = 10  # Number of experiments (days)
avg_cost = simulate_experiment(dummy_policy, E)
print(f"Average cost over {E} experiments (dummy policy): {avg_cost:.2f}")


Average cost over 10 experiments (dummy policy): 624.78
